In [ ]:
FLOP = (2 * 1024 * 1600 * 1600 * 3  # QKVproj = 3*2Ld_m^2
+ 2 * 25 * 1024 * 1024 * 64 # QKT (bs,h,T,d_k) @ (bs,h,T,d_k) = 2hL^2d_k
+ 2 * 25 * 1024 * 1024 * 64 # Attn @ V (bs,h,T,T) @ (bs,h,T,d_k) = 2hL^2d_k
+ 2 * 1024 * 1600 * 1600 # Val @ Wo^T (bs,T,d_m) @ d_m, d_m = 2Ld_m^2
# + 2 * 1024 * 1600 * 6400 * 3 # Up L,d_m @ d_m,d_ff
) * 48 + 2 * 1024 * 1600 * 50256 + 2 * 1024 * 50256 * 1600 # final proj
print(FLOP)

27682406400
